<a href="https://colab.research.google.com/github/ranggarahman/456849-Rangga-Aulia-R-ResponsiJuniorProject/blob/master/Senpro_TI_AUTOML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modul 7 - AutoML

## Instalasi Autogluon
*Modul ini bersumber dari dokumentasi situs Autogluon dengan beberapa modifikasi.*

Pada bagian ini, Anda akan menginstall AutoGluon di local komputer Anda, dalam kasus ini kita akan menggunakan Google Colab sebagai mesin komputasinya. 
Jalankan kode berikut untuk menginstall Autogluon menggunakan package manager PIP.

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

## AutoML dengan kasus tabular dataset
Dalam parktikum ini, kita akan  menggunakan `TabularPredictor` di library AutoGluon untuk memprediksi nilai kolom target berdasarkan kolom lain dalam kumpulan data tabular. Selain data tabular AutoGluon dapat digunakan pada tipe data yang lain seperti citra, teks, audio, dan yang lainnya. Anda diharapkan bisa bereksplorasi dengan tipe data yang lain.

Pastikan AutoGluon tealah diinstal, lalu import `TabularDataset` dan `TabularPredictor` dari AutoGluon. Proses yang akan dilalui diawali dengan proses untuk memuat data lalu melatih model dan membuat prediksi.

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

## Dataset 
kita akan menggunakan dataset dari  [Nature issue 7887](https://www.nature.com/nature/volumes/600/issues/7887): [AI-guided intuition for math theorems](https://www.nature.com/articles/s41586-021-04086-x.pdf). Tujuannya adalah untuk memprediksi *signature* berdasarkan atribut-atribut penjelasnya. Kita mengambil  10 ribu sampel sebagai data latih dan 5 ribu sampel sebagai data uji [data asli](https://github.com/deepmind/mathematics_conjectures/blob/main/knot_theory.ipynb). Dengan sampel yang lebih kecil dari data asli akan mempercepat praktikum ini, namun AutoGluon dapat menangani kumpulan data lengkap jika diinginkan.

Dataset ini dapat langsung dimuat dari URL yang diberikan. `TabularDataset` AutoGluon adalah subkelas pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), sehingga metode `DataFrame` apa pun dapat digunakan pada `TabularDataset`.

Silakan jalankan dan scroll ke kanan untuk melihat seluruh kolom dan label signature dari dataset ini.



In [ ]:
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

Target kita adalah kolom "signature", yang memiliki 18 bilangan bulat unik. Meskipun pandas tidak mengenali tipe data ini dengan benar sebagai kategorikal, AutoGluon akan memperbaiki masalah ini secara otomatis.

In [ ]:
# nama kolom target
label = 'signature'
train_data[label].describe()

## Pelatihan

Kita buat obyek `TabularPredictor` dengan menentukan nama kolom label, lalu melatih dataset dengan `TabularPredictor.fit()`. Kita tidak perlu menentukan parameter lain. AutoGluon akan mengenali ini sebagai tugas klasifikasi multi-kelas, melakukan rekayasa fitur otomatis, melatih beberapa model, dan kemudian menyatukan model untuk membuat predictor akhir.

In [ ]:
predictor = TabularPredictor(label=label).fit(train_data)

Pelatihan model memerlukan waktu beberapa menit atau kurang tergantung pada CPU Anda. Anda dapat membuat pelatihan lebih cepat dengan menentukan argumen `time_limit`. Misalnya, `fit(..., time_limit=60)` akan menghentikan latihan setelah 60 detik. Batas waktu yang lebih tinggi umumnya akan menghasilkan performa prediksi yang lebih baik, dan batas waktu yang terlalu rendah akan mencegah AutoGluon untuk melatih dan menggabungkan serangkaian model yang baik.

Dari output hasil pelatihan di atas, terdapat beberapa model yang secara otomatis dicoba oleh AutoGluon. Autogluon mendapatkan model yang paling baik seperti yang tertampil pada output console.
Semua model (tidak hanya best model) tersimpan pada folder AutoGluon. Anda dapat melihatnya dengan mengklik icon folder di sebelah kiri, Anda akan menemukan model-model yang telah ditrain oleh AutoGluon di folder AutoGluonModels.

Anda dapat melihat sekali lagi urutan model dari hasil pelatihan dengan memanggil method leaderboard.

### Pertanyaan 1:
1. Sebutkan 3 Model yang paling akurat dari beberapa model yang telah dicoba di atas oleh AutoGluon?
2. Model apa yang memiliki proses training yang paling cepat?
3. Model apa yang memiliki proses inferensi yang paling cepat?

In [ ]:
predictor.leaderboard()

Anda dapat memuat model yang telah disimpan menggunakan perintah berikut:

`predictor = TabularPredictor.load("AutogluonModels/ag-xxx-xxx/")`

Path silakan disesuaikan dengan hasil masing-masing

## Prediksi /inferensi

Setelah kita memiliki model yang sesuai dengan dataset pelatihan, kita dapat memuat dataset terpisah untuk digunakan sebagai prediksi dan evaluasi.

In [ ]:
test_data = TabularDataset(f'{data_url}test.csv')

# Buang kolom label atau target pada dataset sebagai berikut
y_pred = predictor.predict(test_data.drop(columns=[label]))

# menampilkan hasil prediksi
y_pred.head()

## Evaluasi

Kita dapat mengevaluasi predictor pada data uji menggunakan fungsi `evaluate()`, yang mengukur seberapa baik kinerja prediktor kita pada data yang tidak digunakan untuk menyesuaikan model.

In [ ]:
predictor.evaluate(test_data, silent=True)

## Menggunakan model yang lain
Proses prediksi dan evaluasi secara default akan menggunakan model dengan performa terbaik yang didapatkan dari data validasi saat proses training. Namun, jika kita ingin menggunakan model yang lain, kita bisa memberikan argumen `model=model_name` pada saat proses prediksi dan evaluasi. 

`model_name` yang dapat digunakan hanya model yang telah diikutsertakan dalam proses training. Untuk melihat model yang dapat digunakan, kita bisa melihat leaderboard seperti yang telah dilakukan sebelumnya atau memanggil method `get_model_name` pada obyek predictor.  

In [ ]:
predictor.get_model_names()


Prediksi dengan menggunakan XGBoost model.

In [ ]:
# Buang kolom label atau target pada dataset sebagai berikut
y_pred = predictor.predict(test_data.drop(columns=[label]),
                           model='XGBoost')

# menampilkan hasil prediksi
y_pred.head()

Prediksi dengan menggunakan XGBoost model.

In [ ]:
predictor.evaluate(test_data, model='XGBoost', silent=True)

# Tugas 1
1. Buatlah model menggunakan AutoML untuk Dataset auto-mpg dataset. 

* Anda bisa mendapatkan dataset pada link berikut: 
https://raw.githubusercontent.com/plotly/datasets/master/auto-mpg.csv
* Model ini merupakan tugas regresi, dengan kolom target `mpg`.
* Secara garis besar langkah-langkah yang akan digunakan sama dengan proses klassifikasi di atas. AutoML akan menentukan secara otomatis tasknya berdasarkan tipe data pada kolom target.
* Langkah-langkah pengerjaan di kerjakan pada sel kode di bawah ini. 




## Deskripsi data:
1. Title: Auto-Mpg Data

2. Sources:
   (a) Origin:  This dataset was taken from the StatLib library which is
                maintained at Carnegie Mellon University. The dataset was 
                used in the 1983 American Statistical Association Exposition.
   (c) Date: July 7, 1993

3. Past Usage:
    -  See 2b (above)
    -  Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning.
       In Proceedings on the Tenth International Conference of Machine 
       Learning, 236-243, University of Massachusetts, Amherst. Morgan
       Kaufmann.

4. Relevant Information:

   This dataset is a slightly modified version of the dataset provided in
   the StatLib library.  In line with the use by Ross Quinlan (1993) in
   predicting the attribute "mpg", 8 of the original instances were removed 
   because they had unknown values for the "mpg" attribute.  The original 
   dataset is available in the file "auto-mpg.data-original".

   "The data concerns city-cycle fuel consumption in miles per gallon,
    to be predicted in terms of 3 multivalued discrete and 5 continuous
    attributes." (Quinlan, 1993)

5. Number of Instances: 398

6. Number of Attributes: 9 including the class attribute

7. Attribute Information:

    1. mpg:           continuous
    2. cylinders:     multi-valued discrete
    3. displacement:  continuous
    4. horsepower:    continuous
    5. weight:        continuous
    6. acceleration:  continuous
    7. model year:    multi-valued discrete
    8. origin:        multi-valued discrete
    9. car name:      string (unique for each instance)

8. Missing Attribute Values:  horsepower has 6 missing values


In [ ]:
# 1. Membuat training data berdasarkan URL yang diberikan

# <<<<<<KODE ANDA DI SINI>>>>>>>>



In [ ]:
# 2. Menentukan kolom target

# <<<<<<KODE ANDA DI SINI>>>>>>>>



In [ ]:
# 3. Fitting model dengan AutoML

# <<<<<<KODE ANDA DI SINI>>>>>>>>




In [ ]:
# 4. Evaluate minimal 3 model terbaik
#    Gunakan data yang sama untuk evaluasi (karena kita tidak punya data testing)

# <<<<<<KODE ANDA DI SINI>>>>>>>>



## Pertanyaan 2
1. Berdasarkan hasil pelatihan dan evaluasi pada data auto-mpg, metric apa yang menggambarkan performa model tersebut? Jelaskan tentang metric tersebut!

2. Sebutkan 3 model terbaik dan score performanya!

3. Pilihlah model yang kiranya paling baik dari sisi performa ataupun waktu inferensinya! Jelaskan mengapa Anda memilih model tersebut!
